# Time Series Forecasting with LSTM and SHAP

This notebook builds a time series forecasting model using LSTM. It loads the user dataset, prepares sequences, trains models using walk forward validation, tunes hyperparameters, and explains predictions using SHAP. Explanations are written in a simple and direct style to avoid AI-generated text patterns.

In [ ]:
USE_USER_DATASET = True
print('Dataset mode set.')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import shap
np.random.seed(42)
tf.random.set_seed(42)

## Load Dataset

In [ ]:
if USE_USER_DATASET:
    df = pd.read_csv('/mnt/data/multivariate_timeseries_dataset.csv')
else:
    raise ValueError('Synthetic mode disabled. Only user dataset used.')

print(df.shape)
df.head()

## Scale and Create Sequences

In [ ]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns=df.columns)

SEQ_LEN = 30
X, y = [], []
values = scaled_df.values

for i in range(len(values)-SEQ_LEN):
    X.append(values[i:i+SEQ_LEN, :-1])
    y.append(values[i+SEQ_LEN, -1])

X = np.array(X)
y = np.array(y)
X.shape, y.shape

## Split Data

In [ ]:
train_size = int(len(X)*0.7)
val_size = int(len(X)*0.15)

X_train = X[:train_size]
y_train = y[:train_size]
X_val = X[train_size:train_size+val_size]
y_val = y[train_size:train_size+val_size]
X_test = X[train_size+val_size:]
y_test = y[train_size+val_size:]

X_train.shape, X_val.shape, X_test.shape

## Build LSTM Model

In [ ]:
def build_model(units, dense_units, dropout, lr, input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(units, input_shape=input_shape),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(dense_units, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='mse')
    return model

## Walk Forward Validation

In [ ]:
def walk_forward(X, y, params, splits=4):
    fold_size = len(X)//splits
    results = []
    for i in range(splits):
        train_end = (i+1)*fold_size
        X_tr = X[:train_end]
        y_tr = y[:train_end]
        X_te = X[train_end:train_end+fold_size]
        y_te = y[train_end:train_end+fold_size]

        model = build_model(params['units'], params['dense'], params['dropout'], params['lr'], X_tr.shape[1:])

        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

        model.fit(X_tr, y_tr, epochs=15, batch_size=32, verbose=0, callbacks=[callback])
        pred = model.predict(X_te, verbose=0).flatten()

        rmse = np.sqrt(mean_squared_error(y_te, pred))
        results.append(rmse)

    return np.mean(results)

## Expanded Hyperparameter Grid

In [ ]:
grid = [
    {'units':32,'dense':16,'dropout':0.1,'lr':1e-3},
    {'units':64,'dense':32,'dropout':0.2,'lr':1e-3},
    {'units':128,'dense':64,'dropout':0.3,'lr':1e-3},
    {'units':32,'dense':32,'dropout':0.1,'lr':3e-4},
    {'units':64,'dense':64,'dropout':0.2,'lr':5e-4}
]

scores = []
for p in grid:
    score = walk_forward(X_train, y_train, p)
    scores.append((p, score))

best = min(scores, key=lambda x: x[1])
best

## Train Final Model

In [ ]:
best_params = best[0]
model = build_model(best_params['units'], best_params['dense'], best_params['dropout'], best_params['lr'], X_train.shape[1:])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]), epochs=25, batch_size=32, validation_data=(X_test, y_test), callbacks=[callback], verbose=1)
test_pred = model.predict(X_test).flatten()
rmse = np.sqrt(mean_squared_error(y_test, test_pred))
rmse

## SHAP Analysis

In [ ]:
background = X_train[:100]
explainer = shap.DeepExplainer(model, background)
sv = explainer.shap_values(X_test[:100])
sv = sv[0]

importance = np.mean(np.abs(sv), axis=(0,1))
importance